In [1]:
import pandas as pd

In [2]:
def clean_data(df):
    """
    Nettoie les données :
    - Convertit les valeurs None en NaN pour une meilleure gestion pandas
    """
    return df.where(pd.notnull(df), None)

def remove_empty_columns(df):
    """
    Supprime les colonnes où toutes les valeurs sont None
    """
    return df.dropna(axis=1, how='all')

def add_region_using_geopandas(df, shapefile_path):
    """
    Ajoute la région en utilisant les informations géographiques
    """
    # Chargement des données géographiques
    regions_df = pd.read_json(shapefile_path)
    regions_df['num_dep'] = regions_df['num_dep'].astype(str)

    # Fusion des données
    merged_df = pd.merge(
        df, 
        regions_df[['num_dep', 'region_name']], 
        left_on='coordinates.deptcode', 
        right_on='num_dep', 
        how='left'
    )
    
    # Drop de la colonne redondante
    merged_df = merged_df.drop(["num_dep"], axis=1)

    # Renommer la colonne liée
    merged_df.rename(columns={"region_name" : "coordinates.region"}, inplace=True)

    return merged_df

In [3]:
df = pd.read_excel("./../data/ base-etablissements residence seniors V1.xlsx")

In [4]:
# Pipeline de traitement
df = clean_data(df)
df = remove_empty_columns(df)
result = add_region_using_geopandas(df, "./../data/departements-region.json")

# Résultat
result.head()

,_id,title,updatedAt,noFinesset,capacity,legal_status,IsEHPAD,IsEHPA,IsESLD,IsRA,...,raPrice.updatedAt,raPrice.PrixF1,raPrice.PrixF1ASH,raPrice.PrixF1Bis,raPrice.PrixF1BisASH,raPrice.PrixF2,raPrice.PrixF2ASH,raPrice.autreTarifPrest,raPrice.prestObligatoire,coordinates.region
0,17.0,Accueil de jour Aux Lucioles,2020-10-04T18:33:19.000Z,010003978,18.0,Privé non lucratif,False,False,False,False,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,Auvergne-Rhône-Alpes
1,35.0,Accueil de jour Lou-Ve-Nou,2020-10-04T18:33:19.000Z,010009066,12.0,Privé non lucratif,False,False,False,False,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,Auvergne-Rhône-Alpes
2,36.0,Accueil de jour Réseau Mnemosis,2020-10-04T18:33:19.000Z,010009157,12.0,Privé non lucratif,False,False,False,False,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,Auvergne-Rhône-Alpes
3,24.0,Accueil de jour autonome l'Entre-Temps,2020-10-04T18:33:19.000Z,010007078,10.0,Public,False,False,False,False,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,Auvergne-Rhône-Alpes
4,19.0,Accueil de jour de Belley,2020-10-04T18:33:19.000Z,010004398,10.0,Privé non lucratif,False,False,False,False,...,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,Auvergne-Rhône-Alpes


In [5]:
result.columns

Index(['_id', 'title', 'updatedAt', 'noFinesset', 'capacity', 'legal_status',
       'IsEHPAD', 'IsEHPA', 'IsESLD', 'IsRA', 'IsAJA', 'IsHCOMPL', 'IsHTEMPO',
       'IsACC_JOUR', 'IsACC_NUIT', 'IsHAB_AIDE_SOC', 'IsCONV_APL', 'IsALZH',
       'IsUHR', 'IsPASA', 'IsPUV', 'IsF1', 'IsF1Bis', 'IsF2', 'prixMin',
       'coordinates.street', 'coordinates.postcode', 'coordinates.deptcode',
       'coordinates.deptname', 'coordinates.city', 'coordinates.phone',
       'coordinates.emailContact', 'coordinates.gestionnaire',
       'coordinates.website', 'coordinates.latitude', 'coordinates.longitude',
       'ehpadPrice._id', 'ehpadPrice.updatedAt', 'ehpadPrice.prixHebPermCs',
       'ehpadPrice.prixHebPermCd', 'ehpadPrice.prixHebPermCsa',
       'ehpadPrice.prixHebPermCda', 'ehpadPrice.prixHebTempCs',
       'ehpadPrice.prixHebTempCd', 'ehpadPrice.prixHebTempCsa',
       'ehpadPrice.prixHebTempCda', 'ehpadPrice.tarifGir12',
       'ehpadPrice.tarifGir34', 'ehpadPrice.tarifGir56',
       'ehpadPr

In [6]:
# Suppression des colonnes inutiles

final_result = result.drop(['updatedAt', 
       'ehpadPrice._id', 'ehpadPrice.updatedAt', 'ehpadPrice.prixHebPermCs',
       'ehpadPrice.prixHebPermCd', 'ehpadPrice.prixHebPermCsa',
       'ehpadPrice.prixHebPermCda', 'ehpadPrice.prixHebTempCs',
       'ehpadPrice.prixHebTempCd', 'ehpadPrice.prixHebTempCsa',
       'ehpadPrice.prixHebTempCda', 'ehpadPrice.tarifGir12',
       'ehpadPrice.tarifGir34', 'ehpadPrice.tarifGir56',
       'ehpadPrice.autrePrestation', 'ehpadPrice.autreTarifPrest',
       'raPrice._id', 'raPrice.updatedAt', 'raPrice.PrixF1',
       'raPrice.PrixF1ASH', 'raPrice.PrixF1Bis', 'raPrice.PrixF1BisASH',
       'raPrice.PrixF2', 'raPrice.PrixF2ASH', 'raPrice.autreTarifPrest',
       'raPrice.prestObligatoire'], axis = 1)

final_result

,_id,title,noFinesset,capacity,legal_status,IsEHPAD,IsEHPA,IsESLD,IsRA,IsAJA,...,coordinates.deptcode,coordinates.deptname,coordinates.city,coordinates.phone,coordinates.emailContact,coordinates.gestionnaire,coordinates.website,coordinates.latitude,coordinates.longitude,coordinates.region
0,17.0,Accueil de jour Aux Lucioles,010003978,18.0,Privé non lucratif,False,False,False,False,True,...,01,AIN,REYRIEUX,04 74 00 85 01,auxlucioles@orange.fr,Association de Gestion Accueil de Jour,None,45.930657,4.814821,Auvergne-Rhône-Alpes
1,35.0,Accueil de jour Lou-Ve-Nou,010009066,12.0,Privé non lucratif,False,False,False,False,True,...,01,AIN,ST TRIVIER DE COURTES,06 71 40 93 06,accueil@fede01.admr.org,Association ADMR des Pays de Bresse,None,46.460191,5.079833,Auvergne-Rhône-Alpes
2,36.0,Accueil de jour Réseau Mnemosis,010009157,12.0,Privé non lucratif,False,False,False,False,True,...,01,AIN,GEX,04 50 20 15 15,lereseaumnemosis@orange.fr,Le Reseau Mnemosis,None,46.331417,6.057783,Auvergne-Rhône-Alpes
3,24.0,Accueil de jour autonome l'Entre-Temps,010007078,10.0,Public,False,False,False,False,True,...,01,AIN,REPLONGES,03 85 33 53 17,direction@ajmarpa.fr,Communauté Communes du Pays Bagé,None,46.312999,4.876204,Auvergne-Rhône-Alpes
4,19.0,Accueil de jour de Belley,010004398,10.0,Privé non lucratif,False,False,False,False,True,...,01,AIN,BELLEY,04 79 81 05 71,adj.belley@ccass.org,Association Santé et Bien-Être,None,45.755062,5.688966,Auvergne-Rhône-Alpes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11742,NaN,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
11743,NaN,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
11744,NaN,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
11745,NaN,None,None,NaN,None,None,None,None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN


In [7]:
final_result.to_csv("./../data/dataset_to_use.csv", header=True, encoding="utf-8", index=False)